In [1]:
import os
import sys
import numpy

sys.path.append(os.path.join(os.environ['HOME'], 'dev/py'))

In [2]:
SO_LABELS = ('X1SPNORB', 'Y1SPNORB', 'Z1SPNORB')
eV = 1/27.211396132

In [3]:
def two_p_splitting(cmo, occa, occb, ps, aoproper="AOPROPER", ao2soint="AO2SOINT", one_electron=False):
    
    SO_LABELS = ('X1SPNORB', 'Y1SPNORB', 'Z1SPNORB')
    
    from scipy.constants import alpha
    prefactor = alpha**2/2
    
    occupied_alpha = cmo.get_columns(occa)
    occupied_beta = cmo.get_columns(occb)
    
    Da = (occupied_alpha*occupied_alpha.T()).unblock()
    Db = (occupied_beta*occupied_alpha.T()).unblock()
    
        
    ls1 = prop.read(*SO_LABELS, filename=aoproper)
    
    if one_electron:
        ls = ls1
    else:
        ls2 = [0.5*(fa - fb) for fa, fb in [twoso.fockab(Da, Db, c, filename=ao2soint) for c in "xyz"]]  
        ls = [one + two for one, two in zip(ls1, ls2)]
    
    p = cmo.unblock()[:, ps]
    plsp = [prefactor*p.T*ls_component*p for ls_component in ls]
    
    V = makeV(plsp)
    
    eigvals = numpy.linalg.eigvals(V)
    er = numpy.sort(eigvals.real)
    
    print er
    split = abs(.5*er[:2].sum() - .25*er[2:].sum())
    
    return split/eV

def makeV(ls):
    V = numpy.zeros((6, 6), dtype='complex', order='F')
    V[:3, :3] = ls[2]
    V[3:, 3:] = -ls[2]
    V[:3, 3:] = ls[0] + 1j*ls[1]
    V[3:, :3] = ls[0] - 1j*ls[1]
    V *= 1j/2
    return V



In [4]:
from dalmisc import sirrst, twoso
from daltools import prop

In [11]:
#system specific
mol = ("Cystein", "cys")
#mol = ("S", "S")
aoproper = "%s/hf_%s.AOPROPER" % mol
ao2soint = "%s/hf_%s.AO2SOINT" % mol
sirius_rst = "%s/hf_%s.SIRIUS.RST" % mol

nocc_alpha = [32] #[3, 2, 2, 0, 1, 0, 0, 0]
nocc_beta  = [32] #[3, 2, 2, 0, 1, 0, 0, 0]

In [12]:
!ls Cystein

AOPROPER	      int2_sto3g.tar.gz		  lr_cc-pVDZ.tar.gz
cc-pVDZ.mol	      int_cc-pVDZ.out		  lr_cc-pVTZ.out
cc-pVTZ.mol	      int_cc-pVDZ.tar.gz	  lr_cc-pVTZ.tar.gz
hf_cc-pVDZ.hso	      int_cc-pVTZ.AO2SOINT	  lr.dal
hf_cc-pVDZ.hso1       int_cc-pVTZ.master.out	  lr_sto3g.AO2SOINT
hf_cc-pVDZ.hso2       int_cc-pVTZ.out		  lr_sto3g.master.AO2SOINT
hf_cc-pVDZ.out	      int_cc-pVTZ.tar.gz	  lr_sto3g.master.debug.out
hf_cc-pVDZ.tar.gz     int.dal			  lr_sto3g.master.out
hf_cc-pVTZ.hso1       int_sto3g.master.debug.out  lr_sto3g.out
hf_cc-pVTZ.out	      int_sto3g.master.out	  lr_sto3g.so255.AO2SOINT
hf_cc-pVTZ.tar.gz     int_sto3g.so000.out	  lr_sto3g.so255.out
hf.dal		      int_sto3g.so255.out	  lr_sto3g.tar.gz
hf_sto3g.AOPROPER     int_sto3g.tar.gz		  peaksplit24.py
hf_sto3g.hso	      lr1.dal			  shield2.dal
hf_sto3g.hso1	      lr1_sto3g.out		  shield2_sto3g.so255.out
hf_sto3g.hso2	      lr1_sto3g.tar.gz		  shield2_sto3g.tar.gz
hf_sto3g.out	      lr2.dal			  shield.dal
hf_sto3g.tar.gz       l

In [13]:

cmo = sirrst.SiriusRestart(name=sirius_rst).cmo



daltools.util.unformatted.SiriusRestart: file Cystein/hf_cys.SIRIUS.RST not found



IOError: 

In [8]:
x, y, z = 9, 15, 23
#x, y, z = 9, 10, 11 #9, 15, 23

In [9]:
print nocc_alpha, nocc_beta

[32] [32]


In [10]:
print cmo.nrow

[9 6 6 2 6 2 2 1]


In [27]:
two_p_splitting(cmo, nocc_alpha, nocc_beta, (x, y, z), aoproper=aoproper, ao2soint=ao2soint)

AssertionError: 

In [120]:
cmorel = sirrst.SiriusRestart(name="S/hfrel_S.SIRIUS.RST").cmo

In [140]:
two_p_splitting(cmorel, nocc_alpha, nocc_beta, (x, y, z), aoproper=aoproper, ao2soint=ao2soint, one_electron=True)

AssertionError: 